# reporting.ipynb


This notebook reads `benchmark_results.csv` produced by `profiler.ipynb` or `main.ipynb`, then:
- Plots runtime vs input size
- Plots peak memory vs input size
- Generates `complexity_report.md`


In [7]:
%pip -q install pandas matplotlib

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

In [3]:
df = pd.read_csv("benchmark_results.csv")
df

,strategy,n_ticks,time_seconds,peak_memory_mb
0,NaiveMovingAverageStrategy,1000,0.002246,76.671875
1,WindowedMovingAverageStrategy(k=10),1000,0.000675,60.593750


## Plot: Runtime vs Input Size

In [4]:
runtime_plot = "runtime_vs_input.png"

plt.figure()
for strat, sub in df.groupby("strategy"):
    sub = sub.sort_values("n_ticks")
    plt.plot(sub["n_ticks"], sub["time_seconds"], marker="o", label=strat)

plt.xlabel("Input size (ticks)")
plt.ylabel("Runtime (seconds)")
plt.title("Runtime vs Input Size")
plt.legend()
plt.tight_layout()
plt.savefig(runtime_plot)
plt.close()

print("Saved", runtime_plot)

Saved runtime_vs_input.png


## Plot: Peak Memory vs Input Size

In [5]:
memory_plot = "memory_vs_input.png"

plt.figure()
for strat, sub in df.groupby("strategy"):
    sub = sub.sort_values("n_ticks")
    plt.plot(sub["n_ticks"], sub["peak_memory_mb"], marker="o", label=strat)

plt.xlabel("Input size (ticks)")
plt.ylabel("Peak memory (MB)")
plt.title("Peak Memory vs Input Size")
plt.legend()
plt.tight_layout()
plt.savefig(memory_plot)
plt.close()

print("Saved", memory_plot)

Saved memory_vs_input.png


## Generate `complexity_report.md`

In [6]:
report_path = Path("complexity_report.md")

lines = []
lines.append("# Complexity Report: Runtime & Space Complexity in Financial Signal Processing\n\n")

lines.append("## Benchmark Table (timeit + memory_profiler)\n\n")
lines.append("| Strategy | Ticks | Runtime (s) | Peak Memory (MB) |\n")
lines.append("|---|---:|---:|---:|\n")
for _, r in df.sort_values(["strategy", "n_ticks"]).iterrows():
    lines.append(f"| {r['strategy']} | {int(r['n_ticks'])} | {r['time_seconds']:.6f} | {r['peak_memory_mb']:.2f} |\n")

lines.append("\n## Scaling Plots\n\n")
lines.append(f"![Runtime vs Input Size]({runtime_plot})\n\n")
lines.append(f"![Peak Memory vs Input Size]({memory_plot})\n\n")

lines.append("## Theoretical Complexity Annotations\n\n")
lines.append("- **NaiveMovingAverageStrategy**: per-tick time **O(i)** (worst **O(n)**) due to `sum(full_history)`; total over n ticks **O(n²)**; space **O(n)** because it stores the full price history.\n")
lines.append("- **WindowedMovingAverageStrategy**: per-tick time **O(1)** using a fixed-size window and running sum; total **O(n)**; space **O(k)** where k is the window size.\n")

lines.append("\n## Narrative Comparison\n\n")
lines.append(
    "The naive implementation recomputes the moving average from scratch each tick by summing the entire history, "
    "so runtime grows superlinearly and becomes impractical as n increases. "
    "The windowed strategy maintains only the last k prices and a running sum, making each update constant-time. "
    "Memory usage for the naive approach grows linearly with n, while the windowed approach stays bounded by k.\n"
)

report_path.write_text(''.join(lines), encoding='utf-8')
print("Generated", report_path)

Generated complexity_report.md
